# Pmax Helper - 共有キー版（社内用）

**APIキー設定不要！すぐ使えます**

## 使い方
1. 下のセルを上から順番に実行
2. LP URLを入力して「動画を生成」ボタンをクリック
3. 完成した動画をダウンロード

⚠️ **注意**: このノートブックには共有APIキーが含まれています。外部に公開しないでください。

## 1. APIキーの設定（自動）

In [ ]:
import os
from google.colab import userdata

# Colabのシークレットキーから取得
FAL_KEY = userdata.get('FAL_KEY')
LAMBDA_SECRET_KEY = "dummy_key"  # 固定値

# 環境変数に設定
os.environ['FAL_KEY'] = FAL_KEY
os.environ['LAMBDA_SECRET_KEY'] = LAMBDA_SECRET_KEY

print("✅ APIキーを設定しました（Colabシークレットから取得）")
print(f"   FAL_KEY: {FAL_KEY[:20]}..." if FAL_KEY else "   ❌ FAL_KEYが見つかりません")

## 2. セットアップ

In [ ]:
# 既存のディレクトリを削除して最新版をクローン
!rm -rf pmax-helper
!git clone https://github.com/KingHippopotamus/pmax-helper.git
%cd pmax-helper

# 最新版を取得（念のため）
!git pull origin main
!echo "✅ 最新コミット:" && git log -1 --oneline
!echo "\n📝 video_generator.py の最終更新:"
!git log -1 --oneline -- backend/services/video_generator.py

%cd backend

# 依存関係をインストール
!pip install -q flask flask-cors requests beautifulsoup4 Pillow fal-client python-dotenv ipywidgets

print("\n✅ セットアップ完了")

## 3. 動画生成ツール

下のセルを実行すると、入力フォームが表示されます。

In [ ]:
import sys
import os

# モジュールキャッシュをクリア（重要！）
for module in list(sys.modules.keys()):
    if 'services' in module or 'fal' in module:  # fal関連も削除
        del sys.modules[module]

sys.path.insert(0, '/content/pmax-helper/backend')

# ⬇️⬇️⬇️ 重要: インポート前に環境変数を設定 ⬇️⬇️⬇️
# 環境変数からAPIキーを取得して、すぐに設定
FAL_KEY = os.getenv('FAL_KEY')
LAMBDA_SECRET_KEY = os.getenv('LAMBDA_SECRET_KEY')

if FAL_KEY:
    os.environ['FAL_KEY'] = FAL_KEY
    print(f"✅ FAL_KEY設定完了: {FAL_KEY[:20]}...")
else:
    print("❌ FAL_KEYが見つかりません")
# ⬆️⬆️⬆️ ここまで移動 ⬆️⬆️⬆️

# この後にインポート（FAL_KEYが既に設定済み）
from services.page_analyzer import PageAnalyzer
from services.video_generator import VideoGenerator
from services.scraper import ImageScraper
from services.image_processor import ImageProcessor
import ipywidgets as widgets
from IPython.display import display, HTML, Video, Image
import requests

# デバッグ: 読み込まれたモジュールのパスを確認
import services.video_generator as vg_module
print(f"\n[DEBUG] video_generator.pyのパス: {vg_module.__file__}")

# ファイルの内容を確認（モデル名が含まれているか）
with open(vg_module.__file__, 'r') as f:
    content = f.read()
    if 'fal-ai/sora-2/image-to-video/pro' in content:
        print("[DEBUG] ✅ Pro版モデルが検出されました")
    elif 'fal-ai/sora-2/image-to-video' in content:
        print("[DEBUG] ⚠️ 通常版モデルが検出されました（古いバージョン）")
print()

# 出力エリア
output = widgets.Output()

# UI要素
url_input = widgets.Text(
    placeholder='https://example.com',
    description='LP URL:',
    layout=widgets.Layout(width='600px')
)

analyze_button = widgets.Button(
    description='🔍 ページを分析',
    button_style='info',
    layout=widgets.Layout(width='150px')
)

generate_button = widgets.Button(
    description='🎬 動画を生成',
    button_style='success',
    layout=widgets.Layout(width='150px'),
    disabled=True
)

# 分析結果編集エリア（初期は非表示）
edit_area = widgets.VBox([])

# 各項目のテキストボックス
product_name_input = widgets.Text(description='商材名:', layout=widgets.Layout(width='600px'))
target_input = widgets.Text(description='ターゲット:', layout=widgets.Layout(width='600px'))
catchphrase_input = widgets.Text(description='キャッチコピー:', layout=widgets.Layout(width='600px'))
benefit1_input = widgets.Text(description='ベネフィット1:', layout=widgets.Layout(width='600px'))
benefit2_input = widgets.Text(description='ベネフィット2:', layout=widgets.Layout(width='600px'))
offer_input = widgets.Text(description='オファー:', layout=widgets.Layout(width='600px'))
cta_input = widgets.Text(description='CTA:', layout=widgets.Layout(width='600px'))

# キャラクター画像プレビュー専用エリア
character_preview = widgets.Output()

# 画像カスタマイズUI
image_url_input = widgets.Text(
    placeholder='https://example.com/character.jpg または 空欄でLP自動抽出',
    description='画像URL:',
    layout=widgets.Layout(width='600px')
)

upload_button = widgets.FileUpload(
    accept='image/*',
    multiple=False,
    description='または画像選択'
)

image_preview = widgets.Output()

# アスペクト比選択ドロップダウン
aspect_ratio_dropdown = widgets.Dropdown(
    options=['16:9 (横長)', '9:16 (縦長)'],
    value='16:9 (横長)',
    description='動画形式:',
    layout=widgets.Layout(width='300px')
)

# グローバル変数
product_info = {}
character_image_url = ""
character_image_data = None  # アップロードされた画像データ
video_url = ""

def on_analyze_clicked(b):
    global product_info, character_image_url
    
    with output:
        output.clear_output()
        print("🔍 ページを分析しています...\n")
        
        try:
            # ページ分析
            analyzer = PageAnalyzer(LAMBDA_SECRET_KEY)
            result = analyzer.analyze_page(url_input.value)
            
            if 'error' in result:
                print(f"\n❌ エラー: {result['error']}")
                return
            
            # 結果を保存
            product_info = result
            character_image_url = result.get('character_image_url', '')
            
            # テキストボックスに値を設定
            product_name_input.value = result.get('product_name', '')
            target_input.value = result.get('target_audience', '')
            catchphrase_input.value = result.get('catchphrase', '')
            benefit1_input.value = result.get('benefit1', '')
            benefit2_input.value = result.get('benefit2', '')
            offer_input.value = result.get('offer', '')
            cta_input.value = result.get('cta_text', '')
            
            # キャラクター画像をプレビュー
            with character_preview:
                character_preview.clear_output()
                if character_image_url:
                    print("🖼️ LP抽出キャラクター画像:")
                    display(HTML(f'<img src="{character_image_url}" width="300">'))
                else:
                    print("⚠️ キャラクター画像が見つかりませんでした")
            
            # 編集エリアを更新
            edit_area.children = [
                widgets.HTML("<h3>📝 分析結果（編集可能）</h3>"),
                character_preview,
                product_name_input,
                target_input,
                catchphrase_input,
                benefit1_input,
                benefit2_input,
                offer_input,
                cta_input
            ]
            
            print("✅ 分析完了！必要に応じて内容を編集してください。")
            generate_button.disabled = False
            
        except Exception as e:
            print(f"\n❌ エラーが発生しました: {str(e)}")
            import traceback
            traceback.print_exc()

def on_image_url_changed(change):
    global character_image_url, character_image_data
    url = change['new'].strip()
    if url:
        # URL入力があればLP抽出結果を上書き
        character_image_url = url
        character_image_data = None  # URLモードに切り替え
        # プレビュー表示
        with image_preview:
            image_preview.clear_output()
            print(f"✅ カスタム画像URL指定: {url[:50]}...")
            display(HTML(f'<img src="{url}" width="200">'))
    else:
        # 空欄の場合はLP抽出結果を使用
        character_image_data = None

def on_file_uploaded(change):
    global character_image_data, character_image_url

    uploaded_files = change['new']
    if uploaded_files:
        # アップロードされた画像データを取得
        filename = list(uploaded_files.keys())[0]
        file_info = uploaded_files[filename]
        character_image_data = file_info['content']

        # URLモードをクリア（アップロードモードに切り替え）
        character_image_url = None

        # プレビュー表示
        with image_preview:
            image_preview.clear_output()
            print(f"✅ 画像アップロード完了: {filename}")
            display(Image(data=character_image_data, width=200))

def on_generate_clicked(b):
    global video_url, product_info
    
    # テキストボックスから最新の値を取得
    product_info.update({
        'product_name': product_name_input.value,
        'target_audience': target_input.value,
        'catchphrase': catchphrase_input.value,
        'benefit1': benefit1_input.value,
        'benefit2': benefit2_input.value,
        'offer': offer_input.value,
        'cta_text': cta_input.value
    })
    
    with output:
        output.clear_output()
        print("🎬 動画を生成しています...（数分かかる場合があります）\n")
        
        try:
            # 画像取得: character_image_dataがあればそれを使用、なければURLからダウンロード
            if character_image_data:
                # ファイルアップロードモード
                print("✅ アップロード画像を使用\n")
                image_data = character_image_data
            elif character_image_url:
                # URLモード（LP自動抽出 or カスタムURL）
                print(f"📥 画像をダウンロード中: {character_image_url[:80]}...")
                response = requests.get(character_image_url, timeout=10)
                response.raise_for_status()
                image_data = response.content
                print(f"✅ 画像ダウンロード完了 ({len(image_data)} bytes)\n")
            else:
                print("❌ キャラクター画像が指定されていません")
                print("LP分析で画像が見つからなかった場合は、上の「画像カスタマイズ」でURLを入力するか、画像をアップロードしてください。")
                return
            
            # 画像を前処理
            print("🖼️ 画像を前処理中...")
            processor = ImageProcessor()
            processed_image = processor.preprocess_image(image_data)
            print("✅ 画像前処理完了\n")
            
            # 編集後の情報でプロンプトを生成
            analyzer = PageAnalyzer(LAMBDA_SECRET_KEY)
            prompt = analyzer._generate_video_prompt(product_info)
            
            # アスペクト比を取得
            selected_ratio = aspect_ratio_dropdown.value.split(' ')[0]  # "16:9" or "9:16"
            print(f"🎞️ アスペクト比: {selected_ratio}\n")
            
            # 動画生成
            print("🎬 AI動画生成を開始...")
            generator = VideoGenerator()
            result = generator.generate_character_video(
                processed_image,
                prompt=prompt,
                aspect_ratio=selected_ratio
            )
            
            if 'error' in result:
                print(f"\n❌ エラー: {result['error']}")
                return
            
            video_url = result.get('video_url', '')
            
            # 結果を表示
            print("\n" + "="*60)
            print("✅ 動画生成完了！\n")
            print(f"📹 動画URL: {video_url}\n")
            
            # 動画を表示
            display(HTML(f'<video width="640" controls><source src="{video_url}" type="video/mp4"></video>'))
            
            # ダウンロードリンク
            display(HTML(f'<br><a href="{video_url}" download="character_video.mp4" target="_blank" style="font-size: 18px;">📥 動画をダウンロード</a>'))
            
        except Exception as e:
            print(f"\n❌ エラーが発生しました: {str(e)}")
            import traceback
            traceback.print_exc()

# イベントハンドラを登録
analyze_button.on_click(on_analyze_clicked)
image_url_input.observe(on_image_url_changed, names='value')
upload_button.observe(on_file_uploaded, names='value')
generate_button.on_click(on_generate_clicked)

# UIを表示
display(widgets.VBox([
    widgets.HTML("<h2>🎬 Pmax Helper - AI動画生成ツール（共有キー版）</h2>"),

    # LP URL入力 + 分析ボタン
    url_input,
    widgets.HBox([analyze_button]),

    # 分析結果編集エリア（動的に表示）
    edit_area,

    # 画像カスタマイズエリア
    widgets.HTML("<hr><h3>📸 キャラクター画像の変更（任意）</h3>"),
    widgets.HTML("<p style='color: #666;'>LP分析で抽出された画像を使う場合は、以下を空欄のままにしてください</p>"),
    image_url_input,
    upload_button,
    image_preview,

    # アスペクト比選択
    widgets.HTML("<hr><h3>🎞️ 動画形式</h3>"),
    aspect_ratio_dropdown,

    # 動画生成ボタン
    widgets.HTML("<hr>"),
    widgets.HBox([generate_button]),
    output
]))

print("\n👆 上のフォームにLP URLを入力して、ボタンをクリックしてください")

---

## トラブルシューティング

### エラーが出る場合
- LP URLが正しいか確認
- セルを上から順番に実行したか確認

### 動画生成に時間がかかる
- fal-ai APIは通常2-5分かかります
- セッションがタイムアウトした場合は最初から再実行